# PoC de RAG on-premises para pesquisa de artigos técnicos da área de Tecnologia da Informação
<br>

**Introdução**<br>
Este projeto implementa um sistema de Geração Aumentada por Recuperação (RAG) para consultas a artigos técnicos na área de TI. O sistema foi desenvolvido para funcionar completamente *on-premises*, sem necessidade de envio de dados para serviços em nuvem, garantindo a privacidade e segurança de informações sensíveis. Por esta razão, modelos relativamente pequenos foram escolhidos para minimizar o consumo de recursos de processamento e de memória, mas também permite a substituição por modelos maiores ou mais especializados conforme a demanda e disponibilidade de recursos.

Outro foco foi o uso de tecnologias e modelos disponíveis gratuitamente a fim de reduzir o impacto financeiro para a implementação da solução, sem gastos com assinatura de soluções comerciais com custo flutuante.

Para fins de Prova de Conceito, precisávamos de uma base de dados de conhecimento disponível publicamente. A base utilizada foi a do Microsoft Learn sobre um recurso de configuração automatizada de dispositivos chamado de [Windows Autopilot](https://learn.microsoft.com/en-us/autopilot/). O recurso é utilizado em grandes corporações para personalizar laptops e desktops rodando sistema operacional Windows com as credenciais do usuário, aplicativos obrigatórios e as restrições da empresa no primeiro uso.

A fonte foi escolhida por permitir fácil exportação de todo o material para PDF, por ser um recurso amplamente usado em grandes corporações e pela estrutura do documento ser primariamente textual, sem imagens.

<img src="https://raw.githubusercontent.com/fabiofaria-git/BIMaster-Proj/refs/heads/main/MSLearn-Autopilot.png" width=30% height=30% />

O PDF resultante foi extraído em 19 de março de 2025 e possui 618 páginas. Como o conteúdo no Microsoft Learn é dinâmico e atualizado regularmente, extrações em datas posteriores podem ter resultados diferentes. Em seguida, foi feito o upload como um dataset do projeto na plataforma de escolha.

A aplicação utiliza embeddings de texto com janelas de 1024 tokens e o índice vetorial FAISS para recuperar informações relevantes de documentos PDF, que são então utilizadas por um modelo de linguagem (LLM) para gerar respostas precisas às consultas dos usuários.
<br><br>

# Instalação das dependências

Devido à demanda de memória, este projeto foi totalmente desenvolvido na plataforma [Kaggle](https://www.kaggle.com/) que, até a data de publicação deste trabalho, oferecia um ambiente Jupyter Notebook com acesso gratuito a 2 GPUs com 15 GB de VRAM cada. Assim sendo, as bibliotecas instaladas nesta etapa são as que ainda não existiam nativamente na plataforma.

Ao utilizar outras plataformas como Google Colab pode ser necessário instalar outras bibliotecas.

- faiss-gpu: para indexação e busca vetorial com aceleração por GPU
- pdfplumber: para extração de texto de documentos PDF
- langchain-community e langchain-huggingface: para integração com modelos transformers de linguagem
- huggingface_hub: para acesso aos modelos do Hugging Face
- huggingface_hub[hf_xet]: acelera download dos modelos

A flag --quiet reduz a quantidade de saída durante a instalação.

In [ ]:
!pip install -U faiss-gpu pdfplumber langchain-community langchain-huggingface huggingface_hub[cli] huggingface_hub[hf_xet] --quiet

# Importação de bibliotecas

In [ ]:
import os
import faiss
import numpy as np
import logging
import json
import torch
import pdfplumber
import transformers
import re
from tqdm import tqdm
from typing import List, Dict, Tuple, Any
from sklearn.preprocessing import normalize
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings
from huggingface_hub import login

# Autenticação no Hugging Face

Para que a chave de API não fique exposta no código, gere uma chave secreta no Hugging Face e adicione-a em uma variável secreta nos Segredos do Kaggle ou do Colab com o nome de "HF_TOKEN".

Comente/Descomente o código abaixo de acordo com a plataforma escolhida.

In [ ]:
# Obter chave secreta de API no Kaggle.
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token = user_secrets.get_secret("HF_TOKEN"))

# Obter chave secreta de API no Google Colab.
# from google.colab import userdata
# hf_token = userdata()
# login(token = userdata.get('HF_TOKEN'))


# Registro de Eventos (Logging)

Logging foi usado em algumas etapas para registrar informações sobre o processo de extração, indexação e busca. Ajuda a diagnosticar problemas quando ocorrerem, além de prover informações sobre o progresso.

In [ ]:
# Configurando o registro de eventos (logging).
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Inicialização de variáveis

Define caminhos e nomes de arquivos para o sistema
* **pdf_folder:** diretório contendo os documentos PDF
* **index_file:** nome do arquivo para o índice FAISS
* **metadata_file:** nome do arquivo para os metadados dos documentos
* **embeddings_model_name:** Nome do modelo para geração de embeddings
* **llm_model_id:** Nome do modelo de LLM para geração das respostas

In [ ]:
pdf_folder = "/kaggle/input/autopilotfullmanual"
index_file = "faiss_index.faiss"
metadata_file = "faiss_metadata.json"

embeddings_model_name = "BAAI/bge-m3"
llm_model_id = "meta-llama/Llama-3.2-3B-Instruct"

Os modelos escolhidos para esta prova de conceito se destacaram pelos seguintes motivos:

**BAAI/bge-m3** (para embeddings)

* **Alto desempenho semântico:** Captura relações semânticas sofisticadas entre termos técnicos;
* **Multilíngue:** Suporta múltiplos idiomas, incluindo português;
* **Otimizado para retrieval:** Desenvolvido especificamente para sistemas de recuperação de informação
* **Eficiência computacional:** Gera embeddings de alta qualidade com requisitos moderados de recursos;
* **Excelente em consultas técnicas:** Particularmente forte em capturar a semântica de documentação de TI;
* **Suporte a contextos longos:** Processa documentos mais extensos (até 8.192 tokens) sem perda significativa de qualidade;
* **Dimensionalidade adequada:** Balanço ideal entre riqueza de representação e eficiência de armazenamento;
* **Boa integração com FAISS**.

**meta-llama/Llama-3.2-3B-Instruct** (para geração)

* **Tamanho compacto:** Com 3B de parâmetros, oferece bom equilíbrio entre qualidade e requisitos de hardware;
* **Fine-tuned para Instruções:** Otimizado para seguir instruções e responder perguntas com base em contexto;
* **Forte em resumos técnicos:** Capacidade de sintetizar informações de múltiplos fragmentos técnicos;
* **Multilíngue:** Suporta ainda mais idiomas que o **BAAI/bge-m3**, incluindo o português;
* **Desempenho competitivo:** Qualidade de resposta comparável a modelos muito maiores;
* **Baixo uso de VRAM**;
* **Inferência rápida**;
* **Formato de resposta estruturado:** Gera respostas bem organizadas, úteis em contexto corporativo;
* **Controle de alucinações:** Tendência a se ater aos fatos presentes nos documentos recuperados.

Outros modelos testados:

**EMBEDDINGS**
* **Alibaba-NLP/gte-Qwen2-1.5B-instruct:** Excelente modelo, porém muito lento e demanda muita memória.
* **sentence-transformers/all-MiniLM-L6-v2:** Extremamente veloz, baixíssimo consumo de memória, porém limitado a 256 tokens e idioma Inglês.
* **sentence-transformers/all-mpnet-base-v2:** Limitado a 384 tokens e idioma inglês.

Na base de dados utilizada neste PoC, *chunking* em fatias menores que 1024 tokens prejudicou o desempenho da busca, uma vez que os tópicos contidos nos documentos são longos, não cabiam em fatiamentos menores que 512 tokens, resultando em perdas semânticas ou respostas incompletas. Por esta razão, modelos pequenos como os acima foram descartados. Eles podem, contudo, ser úteis em outras bases de dados com tópicos mais concisos. 

**GERAÇÃO**
* **deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B**
* **deepseek-ai/DeepSeek-R1-Distill-Qwen-7B**
* **deepseek-ai/DeepSeek-R1-Distill-Llama-8B**

Os modelos DeepSeek foram descartados por serem modelos com *Reasoning/CoT*, o que era desnecessário para esta aplicação. O modelo DeepSeek V3 (sem *Reasoning*) possui demanda de memória extremamente alta para fins deste PoC, e por isso, foi substituído pelo Llama.

# Carregamento dos modelos

Na próxima célula, são definidos o tokenizador para o modelo de LLM pré-treinado (selecionado na célula anterior) usando a classe *AutoTokenizer* da biblioteca *transformers*, e o modelo de geração de texto, utilizando a classe AutoModelForCausalLM.

Para reduzir o consumo de memória, é possível reduzir a precisão dos pesos do LLM definindo o parâmetro *"load_in_8bit"* como *True*. Isso resultará em respostas menos precisas, mas é possível mitigar a deficiência reduzindo a complexidade das perguntas. 

Em seguida, é instanciado o modelo do Hugging Face hub para geração de embeddings de texto utilizando a classe HuggingFaceEmbeddings da biblioteca *langchain_huggingface*. Como parâmetros para a geração de embeddings foram especificadas a utilização de GPU compatível com CUDA e a normalização das embeddings. A normalização é importante para mecanismos de busca baseados em similaridade de cosseno, como o implementado neste projeto usando FAISS, para que a amplitude do vetor não influencie no *score* de similaridade, mas sim, apenas o ângulo entre os vetores.

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(llm_model_id)
llm_model = transformers.AutoModelForCausalLM.from_pretrained(llm_model_id, device_map="auto", load_in_8bit=False)

embedding_model = HuggingFaceEmbeddings(
    model_name=embeddings_model_name,
    model_kwargs={"device": "cuda", # Alternar para "cpu" quando desejar usar a CPU para embeddings.
                 },
    encode_kwargs={"normalize_embeddings": True, # "batch_size": 16  # Para reduzir o consumo de memória, usar tamanhos de lotes menores com modelos maiores.
                  }
)

# Criação de uma classe de documentos personalizada

Nesta etapa definimos uma nova classe de objeto nomeada "Document" que armazenará os pedaços de texto dos documentos e metadados que serão usados para indexação, pesquisa e recuperação mais a frente.

In [ ]:
class Document:
    """Classe para armazenamento do texto dos documentos junto dos metadados."""
    def __init__(self, text: str, metadata: Dict[str, Any]):
        self.text = text
        self.metadata = metadata
    
    def __repr__(self):
        return f"Document(metadata={self.metadata})"

# Carregamento e extração de texto

A função abaixo carrega documentos PDF da pasta informada na variável "pdf_folder", extrai o texto e armazena-os em um objeto da classe Document criada anteriormente. Também são armazenados os metadados que registram o nome do arquivo PDF de origem e o número de páginas extraídas.

A extração é feita por páginas para limitar o consumo de memória. Caso ocorram erros durante o processo de extração, eles são registrados no Logger e impressos na tela.

In [ ]:
def extract_text_from_pdfs(pdf_folder_path: str) -> List[Document]:
    """
    Extract text from all PDF files in a folder, with error handling and progress tracking.
    
    Args:
        pdf_folder_path: Path to folder containing PDF files
        
    Returns:
        List of Document objects containing text and metadata
    """
    documents = []
    pdf_files = [f for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')]
    
    if not pdf_files:
        logger.warning(f"No PDF files found in {pdf_folder_path}")
        return []
    
    logger.info(f"Processing {len(pdf_files)} PDF files from {pdf_folder_path}")
    
    for filename in tqdm(pdf_files, desc="Extracting text from PDFs"):
        pdf_path = os.path.join(pdf_folder_path, filename)
        
        try:
            with pdfplumber.open(pdf_path) as pdf:
                pdf_text = ''
                for page_num, page in enumerate(pdf.pages):
                    try:
                        page_text = page.extract_text() or ""
                        pdf_text += page_text
                    except Exception as e:
                        logger.warning(f"Error extracting text from page {page_num} in {filename}: {e}")
                
                if pdf_text.strip():  # Only add if we have text
                    document = Document(
                        text=pdf_text,
                        metadata={
                            "source": pdf_path,
                            "filename": filename,
                            "page_count": len(pdf.pages)
                        }
                    )
                    documents.append(document)
                else:
                    logger.warning(f"No text extracted from {filename}")
        
        except Exception as e:
            logger.error(f"Error processing PDF {filename}: {e}")
    
    logger.info(f"Successfully extracted text from {len(documents)} of {len(pdf_files)} PDFs")
    return documents

# Chunking

Esta função fatia o documento criado na extração em trechos de 1024 tokens. Por padrão, uma sobreposição de 50 tokens é mantida entre os trechos para preservar o contexto de cada divisão, mas o tamanho dessa sobreposição pode ser facilmente alterada por meio do parâmetro *"overlap_size"*.

A função também atualiza os metadados incluindo um identificador para cada trecho, tamanho, número da palavra inicial e final no documento original para que seja possível recuperar o trecho extraído ou realçá-lo no documento original (não implementado).

In [ ]:
def chunk_text(document: Document, chunk_size: int = 1024, overlap_size: int = 50) -> List[Document]:
    """
    Split document text into chunks based on word count while preserving context with overlap.
    
    Args:
        document: Document object containing text and metadata
        chunk_size: Approximate number of words per chunk
        overlap_size: Number of words to overlap between chunks
        
    Returns:
        List of Document objects representing chunks
    """
    text = document.text
    words = text.split()
    
    if len(words) <= chunk_size:
        # Document is small enough to be a single chunk
        return [document]
    
    chunks = []
    for i in range(0, len(words), chunk_size - overlap_size):
        # Get the words for this chunk
        chunk_words = words[i:i + chunk_size]
        
        if len(chunk_words) < 10:  # Skip very small trailing chunks
            continue
            
        chunk_text = " ".join(chunk_words)
        
        # Create a new Document with updated metadata
        chunk_doc = Document(
            text=chunk_text,
            metadata={
                **document.metadata,
                "chunk_id": len(chunks),
                "chunk_start_word": i,
                "chunk_end_word": i + len(chunk_words),
                "chunk_size": len(chunk_words),
                "is_chunk": True
            }
        )
        
        chunks.append(chunk_doc)
    
    return chunks

# Pipeline para processar PDFs e criar uma base vetorial pesquisável

A função a seguir executa todo o pipeline para transformar uma coleção de PDFs em uma base vetorial pesquisável. Nela, os seguintes passos são realizados:

**Extração de Texto:** Extrai o texto de todos os arquivos PDF em uma pasta especificada.<br>
**Divisão em Chunks:** Divide o texto extraído em trechos menores (por padrão, 1024 tokens) com uma sobreposição entre trechos (por padrão, 50 tokens).<br>
**Geração de Embeddings:** Cria embeddings para cada fatia de texto utilizando o modelo de embeddings pré-treinado especificado anteriormente.<br>
**Criação e Indexação FAISS:** Cria um índice FAISS e adiciona os embeddings a este índice para permitir buscas por similaridade.<br>
**Persistência:** Salva o índice FAISS e os metadados dos documentos em arquivos para uso posterior.<br>

Em resumo, a função extrai o texto de todos os arquivos PDFs na pasta especificada, processa-os, cria um índice FAISS pesquisável e salva os resultados em disco.

Aqui, Logger é usado novamente para mostrar o progresso e eventuais erros no processamento dos dados.

In [ ]:
def process_pdfs_and_create_vectorstore(pdf_folder_path: str,
                                        faiss_index_filename: str = "faiss_index.faiss",
                                        metadata_filename: str = "faiss_metadata.json",
                                        chunk_size: int = 1024,
                                        overlap_size: int = 50):
    """
    Complete pipeline to process PDFs and create a searchable vector store with metadata.

    Args:
        pdf_folder_path: Path to folder containing PDF files
        faiss_index_filename: Filename to save the FAISS index
        metadata_filename: Filename to save the metadata
        chunk_size: Number of tokens per chunk
        overlap_size: Number of tokens to overlap between chunks

    Returns:
        Tuple of (FAISS index, list of document chunks, list of metadata)
    """
    # Step 1: Extract text from PDFs
    documents = extract_text_from_pdfs(pdf_folder_path)
    if not documents:
        logger.error("No documents were successfully processed")
        return None, [], []

    # Step 2: Chunk text into smaller sections
    all_chunks = []
    for doc in documents:
        chunks = chunk_text(doc, chunk_size, overlap_size)
        all_chunks.extend(chunks)

    logger.info(f"Created {len(all_chunks)} chunks from {len(documents)} documents")

    # Step 3: Create embeddings for all chunks
    texts = [chunk.text for chunk in all_chunks]
    metadata = [
        {**chunk.metadata, "text": chunk.text}
        for chunk in all_chunks
    ]

    logger.info("Creating embeddings for all chunks")
    embeddings = []
    for text in tqdm(texts, desc="Creating Embeddings"):  # Wrap texts with tqdm to track progress.
        embedding = embedding_model.embed_documents([text])[0]
        embeddings.append(embedding)
    embeddings = np.array(embeddings).astype('float32')

    # Step 4: Create and save FAISS vector store
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)  # Inner product indexing with normalized vectors is similar to cosine similarity.
    index.add(embeddings)

    faiss.write_index(index, faiss_index_filename)
    with open(metadata_filename, 'w') as f:
        json.dump(metadata, f)

    logger.info(f"FAISS index with {index.ntotal} vectors saved to {faiss_index_filename}")
    logger.info(f"Document metadata saved to {metadata_filename}")

    return index, all_chunks, metadata

# Busca por Similaridade

Com todo o pipeline de ETL pronto, ainda é preciso um mecanismo para recuperação dos documentos. A próxima função realiza a busca de documentos similares a uma dada consulta usando o índice FAISS criado na etapa anterior.

A função:
* Carrega o índice FAISS e os metadados dos documentos.<br>
* Gera os embeddings da consulta utilizando o mesmo modelo de embeddings usado para indexar os documentos.<br>
* Busca no índice FAISS os vetores mais próximos (mais similares) ao embedding da consulta.<br>
* Retorna uma lista dos *top_k* resultados, contendo a pontuação de similaridade, os metadados e o texto de cada documento encontrado.

Vale notar que o tipo de indexação do FAISS (*IndexFlatIP* -- produto interno) com vetores normalizados é similar ao cálculo de similaridade de cosseno.

In [ ]:
def search_similar_documents(query: str, index_path: str, metadata_path: str, top_k: int = 5):
    """
    Search for documents similar to the query.
    
    Args:
        query: Query text to search for
        index_path: Path to the FAISS index file
        metadata_path: Path to the metadata JSON file
        top_k: Number of top results to return
        
    Returns:
        List of tuples (similarity score, document metadata, document text)
    """
    # Load the index
    index = faiss.read_index(index_path)
    
    # Load the metadata
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    
    # Encode the query
    query_embedding = embedding_model.embed_documents([query])
    query_embedding = normalize(np.array(query_embedding), axis=1).astype('float32')
    
    # Search the index
    scores, indices = index.search(query_embedding, top_k)
    
    # Combine results
    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx >= 0 and idx < len(metadata):  # Valid index
            doc_metadata = metadata[idx]
            doc_text = doc_metadata.pop("text", "Text not available")
            results.append((float(score), doc_metadata, doc_text))
    
    return results

# Re-Ranking

Durante testes, foi verificado que os resultados frequentemente ignoravam versóes de software quando eram mencionados. Por exemplo, em uma consulta sobre a compatibilidade e requisitos para uso de um determinado recurso no sistema operacional "Windows 10", os resultados de pesquisa ignoravam a versão 10 e traziam resultados genéricos para qualquer edição do Windows que encontrasse nos documentos.

Dentre os motivos, estão:

* **Relevância dos Tokens:** Números e tokens curtos geralmente recebem menos peso nos modelos de embedding, pois são muito comuns em textos.

* **Diluição de Contexto:** Quando as palavras-chaves de consulta aparecem com muita frequência nos documentos, ela domina o sinal semântico do documento, obscurecendo distinções sutis como "Windows 10".

* **Viés de Pré-Treinamento:** Modelos de embedding podem não ter sido treinados para distinguir versões de produtos ou especificações técnicas como distinções semânticas relevantes.

Como o modelo selecionado é consideravelmente pequeno em número de parâmetros, já era esperado que tivesse dificuldades com nuances desse tipo.

Devido a isso, uma função de busca adicional foi criada para fazer *re-ranking* dos documentos pesquisados, por meio de impulsionamento (*boosting*) dos resultados que contêm a versão explícita do software na *query*, quando existir.

A função de busca híbrida *technical_version_search* é uma implementação de busca por similaridade semântica com filtragem por expressões regulares e técnicas de *boosting* e *re-ranking* dos resultados. Quando o algoritmo detecta determinados padrões de versionamento de software na consulta *(ex: "Windows 10", "iOS 18.2")*, recupera mais documentos iniciais do que o especificado em *top_k* e dá pesos maiores para resultados que contêm correspondências exatas de versão, recalculando as pontuações para cada documento recuperado por similaridade semântica.

Quando o padrão não é detectado na consulta, o algoritmo reverte para a função de busca padrão acima.

A eficiência na recuperação de artigos técnicos teve uma melhora significativa quando este mecanismo de busca híbrido foi implementado. No entanto, modelos de embeddings maiores (ex.: 70B) podem não precisar deste recurso pois tendem a capturar essas nuances semânticas com maior assertividade.

In [ ]:
def technical_version_search(query, index_path, metadata_path, top_k=5):
    """
    Search optimized for technical product version queries
    
    Specifically addresses the problem of product version numbers
    being overlooked in embedding-based search.
    """
    
    # 1. Detect version numbers in query
    version_patterns = re.findall(r'(\w+)\s+(\d+(?:\.\d+)*)', query)
    
    # If no product version is detected in the query, fall back to standard search
    if not version_patterns:
        return search_similar_documents(query, index_path, metadata_path, top_k)
    
    # 2. Load resources (FAISS index and JSON metadata)
    index = faiss.read_index(index_path)
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    
    # 3. Create semantic query embedding
    query_embedding = embedding_model.embed_documents([query])
    query_embedding = normalize(np.array(query_embedding), axis=1).astype('float32')
    
    # 4. Get more initial candidates than needed
    initial_k = min(100, len(metadata))
    scores, indices = index.search(query_embedding, initial_k)
    
    # 5. Get candidate documents
    candidates = []
    for i, idx in enumerate(indices[0]):
        if idx >= 0 and idx < len(metadata):
            text = metadata[idx].pop("text", "")
            candidates.append((float(scores[0][i]), metadata[idx], text))
    
    # 6. Apply version-specific filtering
    filtered_results = []
    
    for score, meta, text in candidates:
        version_match_score = 0
        text_lower = text.lower()
        
        # Check for exact version matches
        for product, version in version_patterns:
            product_lower = product.lower()
            version_pattern = fr"{product_lower}\s+{version}\b"
            
            # If exact version match found, give big boost
            if re.search(version_pattern, text_lower):
                version_match_score = 10.0  # Strong boost for exact version match
            # Otherwise check for product name
            elif product_lower in text_lower:
                version_match_score = 0.5  # Small boost for just product
        
        # Combined score with heavy emphasis on version matching
        combined_score = (score * 0.2) + (version_match_score * 0.8)
        
        # Only include results with some version relevance
        if version_match_score > 0:
            filtered_results.append((combined_score, meta, text))
    
    # If filtering gave us results, use them
    if filtered_results:
        filtered_results.sort(reverse=True, key=lambda x: x[0])
        return filtered_results[:top_k]
    
    # Fall back to regular results
    return candidates[:top_k]

# Processamento dos PDFs

O trecho abaixo executa o processamento completo dos PDFs para criar o índice vetorial.

* Extrai texto dos PDFs na pasta especificada pela variável *pdf_folder*.
* Divide os documentos em pedaços menores.
* Gera embeddings para cada trecho de texto.
* Cria e salva o índice FAISS junto dos metadados.

In [ ]:
# Process PDFs and create vector store
index, chunks, metadata = process_pdfs_and_create_vectorstore(
    pdf_folder,
    faiss_index_filename=index_file,
    metadata_filename=metadata_file
)

# Criação de pipeline de geração de texto

Nesta etapa, o pipeline de geração de texto é inicializado com os seguintes parâmetros:
- max_new_tokens: limite de tokens a gerar (1024)
- temperature: controle de aleatoriedade (0.5)
- do_sample: habilita amostragem probabilística (True)
- truncation: habilita truncamento de textos longos

O número máximo de tokens (1024) junto da habilitação de truncamento foram escolhidos para limitar o consumo de memória, lentidão e para evitar erros ou resultados imprevisíveis. Os modelos escolhidos suportam essa janela escolhida. Como mencionado anteriormente, para os documentos da base de dados teste, janelas menores tiveram resultados com menor qualidade pois muitas das respostas envolvem contextos maiores que 512 tokens.

O parâmetro *temperature* foi ajustado para 0.5 junto com *do_sample* = ***True***, pois neste caso, a intenção é permitir que o modelo retorne respostas moderadamente mais naturais, ao invés de meras cópias dos documentos de origem. Até 0.4, o modelo retornou respostas muito extensas ou prolixas. Já em temperaturas maiores que 0.5, as respostas ignoraram nuances e passos importantes das instruções.

In [ ]:
pipe = transformers.pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=1024,
    temperature=0.5,
    do_sample=True,
    truncation=True
)

llm = HuggingFacePipeline(pipeline=pipe)

# RAG (Retrieval-Augmented Generation)

A função *answer_query* implementa o componente principal do sistema RAG: prover respostas às consultas do usuário.

Parâmetros:
- query: string de consulta do usuário
- index_path: caminho para o índice FAISS
- metadata_path: caminho para o arquivo de metadados
- top_k: número de documentos a recuperar
- debug: se *True*, exibe informações adicionais de depuração (id do documento, nome do arquivo de origem, id do chunk, pontuação no ranking de similaridade, o contexto recuperado, além do Chain-of-Thought, caso modelos com Reasoning sejam utilizados. Por padrão, esta opção é definida como *False* para obtermos uma resposta mais limpa.

Aqui a variável top_k é limitada a 2 documentos para evitar estourar o limite de contexto do modelo de LLM escolhido (o Llama 3.2 3B-Instruct é capaz de processar até 4096 tokens). Modelos superiores têm demanda de memória que excedem as metas deste projeto e utilizar quantização neles poderia resultar em desempenho inferior.

Esta função também passa instruções ao LLM sobre como responder às consultas, atendo-se estritamente ao contexto. Caso a informação solicitada não esteja no contexto, a LLM é instruída a responder que não possui informações suficientes para responder a pergunta.

In [ ]:
def answer_query(query: str, index_path: str, metadata_path: str, top_k: int = 4, debug: bool = True):
    """
    Answer a query using retrieved documents and LLM.
    
    Args:
        query: User query string
        index_path: Path to FAISS index
        metadata_path: Path to metadata file
        top_k: Number of documents to retrieve
        debug: If True, prints additional debugging information
        
    Returns:
        LLM answer based on retrieved context.
    """
    # Retrieve relevant documents
    search_results = technical_version_search(query, index_path, metadata_path, top_k)
    
    # Show debug info if requested
    if debug:
        print(f"Retrieved {len(search_results)} documents:")
        for i, (score, metadata, _) in enumerate(search_results):
            filename = metadata.get("filename", "Unknown")
            chunk_id = metadata.get("chunk_id", "N/A")
            print(f"  Doc {i+1}: Score={score:.2f}, File={filename}, Chunk={chunk_id}")
    
    # Build context from retrieved documents for the LLM
    context = ""
    for _, _, text in search_results:
        context += f"\n{text}\n"
    
    # Create prompt with context and query
    prompt = f"""You are an AI assistant tasked with answering questions based on the provided documents.
Please answer the question based only on the context provided. If the context doesn't contain the 
information needed to answer the question, say "I don't have enough information to answer this question."

Context:
{context}

Question: {query}

Answer:"""
    
    # Get full response from LLM
    full_response = llm(prompt)
    
    # Extract just the answer part from the response
    # Most LLMs will repeat "Answer:" before giving their answer
    if "Answer:" in full_response:
        # Split by "Answer:" and take everything after it
        answer = full_response.split("Answer:", 1)[1].strip()
    else:
        # If no "Answer:" marker is found, try to extract just the generated part
        # This is trickier and less reliable, but we'll do our best
        # Find where our prompt ends and extract everything after that
        if prompt in full_response:
            answer = full_response[len(prompt):].strip()
        else:
            # Last resort: just return the full response
            answer = full_response
    
    return answer

# Interface interativa para o sistema RAG

Por fim, a última função cria uma interface interativa para a sessào de perguntas e respostas. Para encerrar o loop, basta o usuário digitar 'sair'.

In [ ]:
def run_rag_query(faiss_index_filename: str = "faiss_index.faiss",
                  metadata_filename: str = "faiss_metadata.json"):
    
    """Interface interativa para consulta RAG."""
    
    print("Seja bem-vindo ao Sistema de Consultas de TI!\n")
        
    while True:
        query = input("\nO que deseja saber? (ou digite 'sair' para encerrar): ")
        if query.lower() == 'sair':
            break
            
        print("\nBuscando informação...")
        answer = answer_query(query, index_file, metadata_file)
        print("\nResposta:")
        print(answer)

# Run the interactive interface
if __name__ == "__main__":
    run_rag_query(index_file, metadata_file)

# Análise das Respostas

Para a avaliação do algoritmo, foram preparadas perguntas com diferentes níveis de complexidade sobre o conteúdo dos documentos. Algumas são perguntas cujas respostas são endereçadas diretamente no documento. Outras são mais complexas porque as respostas podem estar em tabelas ou ocupar múltiplos trechos do documento.

1. Can you tell me the difference between Autopilot and Autopilot device preparation?
2. What are the requirements for Windows Autopilot device preparation?
3. How corporate identifiers are set?
4. Is Windows 10 devices compatible with Windows Autopilot Device Preparation?
5. What is the process for a successful registration of a device?

As perguntas foram feitas com três níveis de temperatura diferentes: 0.5, 0.2, 0.1. As respostas para cada caso e as observações estão no arquivo [CompareResults.xlsx](https://github.com/fabiofaria-git/BIMaster-Proj/raw/refs/heads/main/CompareResults.xlsx) no repositório.

- A variação de temperatura não afetou consideravelmente o resultado com os modelos escolhidos. 

- O modelo respondeu corretamente às perguntas 2, 3, 4 e 5 em todos os níveis de temperatura testados.

- As respostas para a pergunta 1 foram parcialmente corretas e incompletas.

A baixa qualidade nas respostas da pergunta 1 evidencia uma deficiência da solução proposta neste PoC: a resposta para a pergunta está em uma longa tabela.

![Tabela com as diferenças solicitadas na pergunta 1](https://raw.githubusercontent.com/fabiofaria-git/BIMaster-Proj/refs/heads/main/AutopilotvsDevicePrep.png)

PDFs não armazenam tabelas como estruturas semânticas, apenas como posicionamento de texto. Há uma perda de estrutura relacional em tabelas se elas não forem tratadas separadamente quando detectadas. Esta deficiência já era prevista. Porém, como os documentos de instruções a equipes de suporte de T.I. raramente contêm tabelas e a implementação de uma solução aumentaria substancialmente a complexidade de todo o pipeline, optou-se por não implementar tal funcionalidade.

# Conclusões

Este PoC demonstra que Large Language Models relativamente pequenos e gratuitos podem viabilizar financeiramente projetos de Inteligência Artificial em ambientes corporativos que não permitem uso de serviços em núvem, mas ainda assim trazer resultados satisfatórios.

Cabe notar que há modelos mais complexos tanto de geração de texto quanto de embeddings que podem melhorar a versatilidade da solução e a qualidade das respostas. Entretanto, um dos maiores entraves no uso desses modelos maiores é o consumo de memória e o tempo de processamento das respostas. Portanto, a relação custo/benefício precisa ser avaliada caso a caso de acordo com os recursos disponíveis.

Dito isto, este projeto permite algumas melhorias com aumento modesto de demanda de memória e de processamento:

1) Implementação de pipeline multimodal que reconheça tabelas, imagens, legendas e URLs;
2) Separação do pipeline ETL do pipeline RAG;
3) *Semantinc chunking* e *Contextual Chunk Headers (CCH)*.

A primeira permitiria extração e armazenamento de imagens, tabelas, legendas e URLs em blocos, como metadados de cada trecho do documento original, garantindo a preservação das relações semânticas nas buscas e consultas. Isso também permitiria que as respostas incluíssem as tabelas e imagens dos documentos de origem.
    
A segunda melhoria proposta resolveria outra deficiência da versão atual desta aplicação: como a cada execução do pipeline, a base vetorial é recriada, há uma variância no processo de chunking e indexação que faz com que a consistência das respostas do modelo variem ligeiramente entre uma execução e outra. Para um ambiente de produção, o recomendável seria separar esses dois pipelines para executar o processo de extração, chunking e indexação em base vetorial somente quando houverem alterações nos documentos da base de dados ou em intervalos regulares (por exemplo, quizenalmente, semanalmente ou mensalmente), garantindo maior consistência nas consultas.

Já a terceira sugestão melhoraria principalmente a recuperação dos documentos mais relevantes. A técnica de *Contextual Chunk Headers (CCH)* usa o modelo de LLM para criar resumos de cada trecho do documento original e armazená-los como metadados antes da criação de embeddings, melhorando a eficácia na recuperação dos dados. Já a técnica de *Semantic Chunking* consiste em fracionar o texto em trechos que mantenham a coesão do contexto, mantendo secções que falam de um mesmo assunto no mesmo *chunk*. *CCH* e *Semântic Chunking* são técnicas bastante custosas computacionalmente e, por esta razão, foram deixadas de fora deste PoC, mas podem ser implementadas em conjunto com a separação do pipeline de ETL e executadas apenas ocasionalmente, minimizando o impacto financeiro desta solução, caso o custo/benefício justificar.